In [1]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving headphone.csv to headphone.csv


In [2]:
import io

df = pd.read_csv(io.BytesIO(uploaded['headphone.csv']))
df.ReviewBody = df.ReviewBody.str.lower()
df = df.dropna()
print(df)

                                             ReviewBody  ReviewStar
0     no doubt it has a great bass and to a great ex...           3
1     this  earphones are unreliable, i bought it be...           1
2     i bought itfor 999,i purchased it second time,...           4
3     its sound quality is adorable. overall it was ...           1
4     its awesome... good sound quality & 8-9 hrs ba...           5
...                                                 ...         ...
4995  sound quality is superb..battery life is good....           5
4996  nice earphone........ good bt connectivity. an...           5
4997  stopped working.... sound is breaking. pls gui...           1
4998  awesome product... worth the price and the sou...           5
4999  noise cancellation is too poor i ordered 2 pai...           1

[5000 rows x 2 columns]


In [3]:
import nltk
nltk.download('stopwords')
import spacy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def generate_candidates(review):
  candidates = []
  current = []
  for tok in review:
    if tok.is_stop or tok.is_punct:
      if current:
        candidates.append(current)
        current = []
    else:
      if tok.is_alpha:
        current.append(tok)
  if current:
    candidates.append(current)
  return candidates


In [5]:
nlp = spacy.load("en_core_web_sm")
candidates = []
for row in df.itertuples():
  text = nlp(row.ReviewBody)
  candidates.extend(generate_candidates(text))
print(candidates)

[[doubt], [great, bass], [great, extent, noise, cancellation], [decent, sound, clarity], [mindblowing, battery], [following, dissapointed], [tried], [lot], [range], [pain], [ear], [conical], [wires], [bit, long], [makes], [odd], [pouch], [worst], [low, quality], [distoring, mic], [person, keeps, complaining], [voice], [earphones], [unreliable], [bought], [days], [right], [ear, buds, got, cracked, automatically], [got, divided], [parts], [sound, quality], [good], [ok], [thing, bass], [good], [boat], [proof], [attached, picsplease], [think], [unreliable], [bought, itfor, purchased], [second, time], [gifted], [brother], [quality], [connect, devices], [connect, laptop], [mobile, listening, muaic], [lappy], [u], [switches], [mobile], [switch], [device], [good], [price], [stand], [hours], [continously], [hours], [battery, life], [pretty], [ear], [sound, quality], [adorable], [overall], [good], [weeks], [stopped, working], [powered], [claiming], [warrenty], [undergoing], [awesome], [good, sou

In [6]:
unique_words={}
#print(type(unique_words))
for candidate in candidates:
  for word in candidate:
    if word.text not in unique_words.keys():
      unique_words[word.text] = 1
    else:
      unique_words[word.text] += 1
print(unique_words)

{'doubt': 24, 'great': 441, 'bass': 940, 'extent': 7, 'noise': 429, 'cancellation': 358, 'decent': 132, 'sound': 2230, 'clarity': 84, 'mindblowing': 3, 'battery': 1270, 'following': 2, 'dissapointed': 6, 'tried': 40, 'lot': 65, 'range': 364, 'pain': 52, 'ear': 398, 'conical': 3, 'wires': 47, 'bit': 139, 'long': 218, 'makes': 44, 'odd': 6, 'pouch': 12, 'worst': 154, 'low': 185, 'quality': 2156, 'distoring': 1, 'mic': 147, 'person': 86, 'keeps': 30, 'complaining': 8, 'voice': 211, 'earphones': 496, 'unreliable': 7, 'bought': 235, 'days': 226, 'right': 130, 'buds': 70, 'got': 349, 'cracked': 2, 'automatically': 70, 'divided': 2, 'parts': 2, 'good': 2869, 'ok': 137, 'thing': 119, 'boat': 567, 'proof': 18, 'attached': 5, 'picsplease': 1, 'think': 72, 'itfor': 1, 'purchased': 98, 'second': 37, 'time': 296, 'gifted': 4, 'brother': 7, 'connect': 91, 'devices': 37, 'laptop': 29, 'mobile': 90, 'listening': 99, 'muaic': 1, 'lappy': 3, 'u': 220, 'switches': 9, 'switch': 38, 'device': 105, 'price':

In [7]:
for word in unique_words.keys():
  word=nlp(word)

In [8]:
import numpy as np
word_df = pd.DataFrame(
    np.zeros((len(unique_words), len(unique_words)), dtype=int),
    index=unique_words,
    columns=unique_words)
for candidate in candidates:
    for w1 in candidate:
        for w2 in candidate:
          if w1.text!=w2.text:
            word_df.loc[w1.text, w2.text] += 1
for word in unique_words.keys():
  word_df.loc[word, word] = unique_words[word]
print(word_df)

            doubt  great  bass  extent  noise  cancellation  decent  sound  \
doubt          24      0     0       0      0             0       0      0   
great           0    441    25       2      7             8       0     57   
bass            0     25   940       0      4             7       6     35   
extent          0      2     0       7      2             2       0      0   
noise           0      7     4       2    429           271       1     10   
...           ...    ...   ...     ...    ...           ...     ...    ...   
mesmerise       0      0     0       0      0             0       0      0   
experiance      0      0     0       0      0             0       0      0   
worthful        0      0     0       0      0             0       0      0   
yoyr            0      0     0       0      0             0       0      0   
manny           0      0     0       0      0             0       0      0   

            clarity  mindblowing  ...  itgot  thx  vth  hadblue

In [9]:
degree={}
for col in word_df.columns:
  degree[col]=word_df[col].sum()
print(degree)


{'doubt': np.int64(34), 'great': np.int64(874), 'bass': np.int64(1700), 'extent': np.int64(20), 'noise': np.int64(1052), 'cancellation': np.int64(888), 'decent': np.int64(254), 'sound': np.int64(5045), 'clarity': np.int64(176), 'mindblowing': np.int64(5), 'battery': np.int64(2932), 'following': np.int64(4), 'dissapointed': np.int64(10), 'tried': np.int64(53), 'lot': np.int64(90), 'range': np.int64(693), 'pain': np.int64(94), 'ear': np.int64(860), 'conical': np.int64(5), 'wires': np.int64(73), 'bit': np.int64(322), 'long': np.int64(445), 'makes': np.int64(92), 'odd': np.int64(12), 'pouch': np.int64(20), 'worst': np.int64(285), 'low': np.int64(375), 'quality': np.int64(5047), 'distoring': np.int64(2), 'mic': np.int64(236), 'person': np.int64(128), 'keeps': np.int64(73), 'complaining': np.int64(15), 'voice': np.int64(428), 'earphones': np.int64(1017), 'unreliable': np.int64(10), 'bought': np.int64(297), 'days': np.int64(309), 'right': np.int64(279), 'buds': np.int64(164), 'got': np.int64(

In [10]:
word_score = {}
for word in unique_words.keys():
  word_score[word] = degree[word]/unique_words[word]
print(word_score)

{'doubt': np.float64(1.4166666666666667), 'great': np.float64(1.981859410430839), 'bass': np.float64(1.8085106382978724), 'extent': np.float64(2.857142857142857), 'noise': np.float64(2.4522144522144522), 'cancellation': np.float64(2.4804469273743015), 'decent': np.float64(1.9242424242424243), 'sound': np.float64(2.2623318385650224), 'clarity': np.float64(2.0952380952380953), 'mindblowing': np.float64(1.6666666666666667), 'battery': np.float64(2.308661417322835), 'following': np.float64(2.0), 'dissapointed': np.float64(1.6666666666666667), 'tried': np.float64(1.325), 'lot': np.float64(1.3846153846153846), 'range': np.float64(1.9038461538461537), 'pain': np.float64(1.8076923076923077), 'ear': np.float64(2.1608040201005023), 'conical': np.float64(1.6666666666666667), 'wires': np.float64(1.553191489361702), 'bit': np.float64(2.316546762589928), 'long': np.float64(2.041284403669725), 'makes': np.float64(2.090909090909091), 'odd': np.float64(2.0), 'pouch': np.float64(1.6666666666666667), 'wo

In [11]:
new_candidates = []
for candidate in candidates:
  st=""
  for word in candidate:
    if st=="":
      st +=word.text
    else:
      st += " " + word.text
  new_candidates.append(st)
print(new_candidates)
candidates = new_candidates

['doubt', 'great bass', 'great extent noise cancellation', 'decent sound clarity', 'mindblowing battery', 'following dissapointed', 'tried', 'lot', 'range', 'pain', 'ear', 'conical', 'wires', 'bit long', 'makes', 'odd', 'pouch', 'worst', 'low quality', 'distoring mic', 'person keeps complaining', 'voice', 'earphones', 'unreliable', 'bought', 'days', 'right', 'ear buds got cracked automatically', 'got divided', 'parts', 'sound quality', 'good', 'ok', 'thing bass', 'good', 'boat', 'proof', 'attached picsplease', 'think', 'unreliable', 'bought itfor purchased', 'second time', 'gifted', 'brother', 'quality', 'connect devices', 'connect laptop', 'mobile listening muaic', 'lappy', 'u', 'switches', 'mobile', 'switch', 'device', 'good', 'price', 'stand', 'hours', 'continously', 'hours', 'battery life', 'pretty', 'ear', 'sound quality', 'adorable', 'overall', 'good', 'weeks', 'stopped working', 'powered', 'claiming', 'warrenty', 'undergoing', 'awesome', 'good sound quality', 'hrs battery life',

In [12]:
phrase_score = {}
for candidate in candidates:
  score = 0
  words = candidate.split(" ")
  for word in words:
    score += word_score[word]
  phrase_score[candidate] = score
print(phrase_score)

{'doubt': np.float64(1.4166666666666667), 'great bass': np.float64(3.790370048728711), 'great extent noise cancellation': np.float64(9.77166364716245), 'decent sound clarity': np.float64(6.281812358045542), 'mindblowing battery': np.float64(3.9753280839895018), 'following dissapointed': np.float64(3.666666666666667), 'tried': np.float64(1.325), 'lot': np.float64(1.3846153846153846), 'range': np.float64(1.9038461538461537), 'pain': np.float64(1.8076923076923077), 'ear': np.float64(2.1608040201005023), 'conical': np.float64(1.6666666666666667), 'wires': np.float64(1.553191489361702), 'bit long': np.float64(4.357831166259652), 'makes': np.float64(2.090909090909091), 'odd': np.float64(2.0), 'pouch': np.float64(1.6666666666666667), 'worst': np.float64(1.8506493506493507), 'low quality': np.float64(4.367936117936118), 'distoring mic': np.float64(3.6054421768707483), 'person keeps complaining': np.float64(5.7967054263565885), 'voice': np.float64(2.028436018957346), 'earphones': np.float64(2.0

In [13]:
phrase_score_sorted=dict(sorted(phrase_score.items(), key=lambda item: item[1], reverse=True))
print(phrase_score_sorted)

{'product ki warrenty kamse kam mahine ho na jaruri hai mahineme iseme problem hu wa hai': np.float64(168.85185537824833), 'ke time bat krne pr srrrr srrr ki awaj aa rhi': np.float64(102.54391891891892), 'oxmmm yr gjb k base hai battery lfe mstsound quality oxmmm value': np.float64(83.08850540586248), 'mujhe mila kuch time use krne ke baad hi kuch': np.float64(68.72383859763377), 'product ko bina tension bahot hi acha peice hai': np.float64(56.02946674537584), 'amazing earphone best sounds qualityand nice appearancealso lite weightsgood battery': np.float64(43.4538334073425), 'helmet k andar nhi pahen isey khareeda': np.float64(43.4), 'matalab skullcandy ke inked se bhi supper': np.float64(41.964285714285715), 'supar bassgood bettry m bluetooh rangeanthig good comfeble': np.float64(41.01537231599971), 'sound dono best sports m base nhi hai bus': np.float64(40.735090648587985), 'ek earbud kaam krna band kr dia': np.float64(38.96212121212122), 'mic problem hu wa hai': np.float64(30.80283

In [14]:
potential_aspects=pd.Series(phrase_score_sorted)
print(potential_aspects)

product ki warrenty kamse kam mahine ho na jaruri hai mahineme iseme problem hu wa hai    168.851855
ke time bat krne pr srrrr srrr ki awaj aa rhi                                             102.543919
oxmmm yr gjb k base hai battery lfe mstsound quality oxmmm value                           83.088505
mujhe mila kuch time use krne ke baad hi kuch                                              68.723839
product ko bina tension bahot hi acha peice hai                                            56.029467
                                                                                             ...    
itgot                                                                                       1.000000
mesmerise                                                                                   1.000000
experiance                                                                                  1.000000
worthful                                                                                   

In [15]:
pd.options.display.max_rows=12000
#print(potential_aspects)
filtered_aspects = potential_aspects[
    potential_aspects.index.str.split().str.len() <= 3
]

print(filtered_aspects.head(50))


quality ki efficiency                      15.340909
sab thik hai                               12.928571
downlod avee player                        11.833333
running ke jogging                         10.748662
secondary earphone incase                  10.153652
defected peice bluetooth                    9.968553
tracking status button                      9.933333
boat stone speakers                         9.671252
gon na write                                9.534091
gon na receive                              9.529762
gon na feel                                 9.391990
created little discomfort                   9.312315
gon na face                                 9.309211
having mi sports                            9.240566
theatre noise cancellation                  9.182661
valuables bluetooh earphones                9.050403
basseasy connectivityperfectly fitsgood     9.000000
ye mstttt hit                               9.000000
headphonealso bestt paart                   9.

In [16]:
aspects=["bluetooth","bass","connectivity","sound","battery","price"]

In [17]:
def find_aspect(doc):
  for token in doc:
    if token.text in sen.keys():
      asp = token.text
      for child in token.children:
        if child.pos_ == "ADJ":
          key = child.text
          negated = False
          for grandchild in child.children:
            if grandchild.dep_ == "neg":
              negated = True
          for child in token.children:
            if child.dep_ == "neg":
              negated = True
          if key not in sen[asp]:
            sen[asp][key] = {"count": 1, "negated": 0 }
          else:
            sen[asp][key]["count"] += 1
          if negated:
            sen[asp][key]["negated"] += 1

In [18]:
sen={}
for aspect in aspects:
  sen[aspect]={}
print(sen)

{'bluetooth': {}, 'bass': {}, 'connectivity': {}, 'sound': {}, 'battery': {}, 'price': {}}


In [19]:
for row in df.itertuples():
  doc = nlp(row.ReviewBody)
  find_aspect(doc)

In [20]:
print(sen)

{'bluetooth': {'previous': {'count': 1, 'negated': 0}, 'good': {'count': 3, 'negated': 0}, 'nice': {'count': 2, 'negated': 0}, 'best': {'count': 4, 'negated': 0}, 'perfect': {'count': 1, 'negated': 0}, 'free': {'count': 1, 'negated': 0}, 'ok': {'count': 1, 'negated': 0}, 'overall': {'count': 1, 'negated': 0}, 'automatic': {'count': 1, 'negated': 0}, 'fast': {'count': 1, 'negated': 0}, 'other': {'count': 2, 'negated': 0}, 'awesome': {'count': 1, 'negated': 0}, 'dual': {'count': 1, 'negated': 0}}, 'bass': {'great': {'count': 23, 'negated': 0}, 'punchy': {'count': 2, 'negated': 0}, 'treble': {'count': 10, 'negated': 0}, 'bass': {'count': 1, 'negated': 0}, 'much': {'count': 9, 'negated': 0}, 'superb': {'count': 5, 'negated': 0}, 'awesome': {'count': 14, 'negated': 0}, 'deep': {'count': 16, 'negated': 2}, 'more': {'count': 9, 'negated': 0}, 'muddy': {'count': 1, 'negated': 0}, 'amazing': {'count': 11, 'negated': 0}, 'nice': {'count': 10, 'negated': 0}, 'good': {'count': 67, 'negated': 2}, '

In [21]:
## aspect frequency
for key in sen.keys():
  sum=0
  for key2 in sen[key].keys():
    sum += sen[key][key2]["count"]
  print(sum)

20
338
33
320
110
81


In [22]:
import nltk
nltk.download("sentiwordnet")
nltk.download("wordnet")

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [23]:
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn

In [24]:
def sentiwordnet_score(adj):
  synsets = list(swn.senti_synsets(adj, pos ="a"))
  if not synsets:
    return 0
  score = 0.0
  for syn in synsets:
    score = score + syn.pos_score() - syn.neg_score()
  return score / len(synsets)

In [25]:
aspect_score = {}
for aspect in aspects:
  aspect_score[aspect] = 0

In [26]:
for aspect in aspects:
  total_count = 0
  s = 0.0
  for adj, data in sen[aspect].items():
    base_score = sentiwordnet_score(adj)
    s += ((data["count"]-2*data["negated"])*base_score)
    total_count+= data["count"]
  aspect_score[aspect] = s/total_count


In [27]:
print(aspect_score)

{'bluetooth': 0.314047619047619, 'bass': 0.27234681732999794, 'connectivity': 0.16294642857142858, 'sound': 0.3373756055627379, 'battery': 0.18251177743824804, 'price': 0.1378350301961413}


**ANALYSING LOWER RATED REVIEWS:**

In [28]:
df2=df[df["ReviewStar"]<4]
print(df2.head(20))

                                           ReviewBody  ReviewStar
0   no doubt it has a great bass and to a great ex...           3
1   this  earphones are unreliable, i bought it be...           1
3   its sound quality is adorable. overall it was ...           1
5   after 11 days, the charging port isn't working...           1
7   after one month usage reviewing this .pros-1- ...           3
9   the product durability is 1 month..i just lost...           1
11  what on earth is the use of buying such produc...           1
13          product stopped working in a month time\n           1
14  right speaker is not working at all. useless p...           1
15  received this boat 255 earphones yesterday eve...           3
17  the product is a good buy if you are looking f...           3
18  below average product. not happy at all.pros :...           1
19  he only problem is that as earpieces are of me...           3
23   rightside speaker stop working within 20 days.\n           1
26  defect

In [29]:
sen2={}
for aspect in aspects:
  sen2[aspect]={}
print(sen2)

{'bluetooth': {}, 'bass': {}, 'connectivity': {}, 'sound': {}, 'battery': {}, 'price': {}}


In [30]:
def find_aspect(doc):
  for token in doc:
    if token.text in sen2.keys():
      asp = token.text
      for child in token.children:
        if child.pos_ == "ADJ":
          key = child.text
          negated = False
          for grandchild in child.children:
            if grandchild.dep_ == "neg":
              negated = True
          for child in token.children:
            if child.dep_ == "neg":
              negated = True
          if key not in sen2[asp]:
            sen2[asp][key] = {"count": 1, "negated": 0 }
          else:
            sen2[asp][key]["count"] += 1
          if negated:
            sen2[asp][key]["negated"] += 1

In [31]:
for row in df2.itertuples():
  doc = nlp(row.ReviewBody)
  find_aspect(doc)

In [32]:
print(sen2)

{'bluetooth': {'previous': {'count': 1, 'negated': 0}, 'ok': {'count': 1, 'negated': 0}, 'dual': {'count': 1, 'negated': 0}, 'other': {'count': 1, 'negated': 0}}, 'bass': {'great': {'count': 7, 'negated': 0}, 'treble': {'count': 1, 'negated': 0}, 'bass': {'count': 1, 'negated': 0}, 'much': {'count': 4, 'negated': 0}, 'nice': {'count': 2, 'negated': 0}, 'dynamic': {'count': 1, 'negated': 0}, 'enjoyable': {'count': 1, 'negated': 0}, 'high': {'count': 3, 'negated': 0}, 'good': {'count': 9, 'negated': 2}, 'decent': {'count': 2, 'negated': 0}, 'extraordinary': {'count': 1, 'negated': 0}, 'lousy': {'count': 1, 'negated': 0}, 'artificial': {'count': 1, 'negated': 0}, 'unnecessary': {'count': 1, 'negated': 0}, 'bloated': {'count': 1, 'negated': 0}, 'many': {'count': 1, 'negated': 0}, 'sound': {'count': 2, 'negated': 0}, 'frank': {'count': 1, 'negated': 0}, 'overall': {'count': 1, 'negated': 0}, 'deep': {'count': 2, 'negated': 0}, 'more': {'count': 2, 'negated': 0}, 'awesome': {'count': 1, 'neg

In [33]:
aspect_score2 = {}
for aspect in aspects:
  aspect_score2[aspect] = 0

In [34]:
for aspect in aspects:
  total_count = 0
  s = 0.0
  for adj, data in sen2[aspect].items():
    base_score = sentiwordnet_score(adj)
    s += ((data["count"]-2*data["negated"])*base_score)
    total_count+= data["count"]
  aspect_score2[aspect] = s/total_count

In [35]:
print(aspect_score2)

{'bluetooth': 0.01822916666666667, 'bass': 0.16072573260073264, 'connectivity': -0.04199862637362636, 'sound': 0.09071026234567903, 'battery': -0.16731132557189543, 'price': 0.04445346320346322}
